## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/great-learning/computer-vision-project2/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
import pandas as pd
import numpy as np

In [0]:
labels = pd.read_csv('labels.csv')

### Print the count of each category of Dogs given in the dataset



In [10]:
labels['breed'].value_counts()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
golden_retriever         67
komondor                 67
brabancon_griffon        67
eskimo_dog               66
briard                   66
Name: breed, Length: 120, dtype: int64

### Get one-hot encodings of labels

In [0]:
targets = pd.Series(labels['breed'])

In [0]:
one_hot_labels = np.asarray(pd.get_dummies(targets, sparse=True))

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
from tqdm import tqdm
import cv2

In [0]:
x_train = []
y_train = []
img_rows = 128
img_cols = 128

In [15]:
for f, img in tqdm(labels.values):
  train_img = cv2.imread('./train/{}.jpg'.format(f),1)
  train_img_resize = cv2.resize(train_img, (img_rows, img_cols))
  x_train.append(train_img_resize)
  y_train.append(img)

100%|██████████| 10222/10222 [00:31<00:00, 324.85it/s]


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
x_train_data = np.asarray(x_train)/255

In [0]:
y_train_data = np.asarray(pd.get_dummies(y_train, sparse=True))

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_train, x_val, y_train, y_val= train_test_split(x_train_data, y_train_data, test_size=0.25, random_state=0)

In [20]:
x_val.shape

(2556, 128, 128, 3)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
samples = pd.read_csv('sample_submission.csv')

In [0]:
test_img = samples['id']

Run the below code to load the test image files in x_test_feature

In [23]:
x_test_feature = []
i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 0)
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test_feature.append(img_resize)

100%|██████████| 10357/10357 [00:14<00:00, 693.53it/s]


Normalize the test data and convert it into 4 dimensions

In [0]:
x_test_data = np.asarray(x_test_feature)/255

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [25]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf

In [0]:
#Clear any previous model from memory
tf.keras.backend.clear_session()

#Initialize model
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Reshape((128,128,3),input_shape=(128,128,3)))

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(5,5), activation='relu'))

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'))

#Add Dense Layers after flattening the data
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(256, activation='relu'))

#Add Output Layer
model.add(tf.keras.layers.Dense(120, activation='softmax'))

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 124, 124, 64)      4864      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 122, 122, 32)      18464     
_________________________________________________________________
flatten (Flatten)            (None, 476288)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               121929984 
_________________________________________________________________
dense_1 (Dense)              (None, 120)               30840     
Total params: 121,984,152
Trainable params: 121,984,152
Non-trainable params: 0
__________________________________________

### Use batch_size = 128 and epochs = 10 and execute the model

In [30]:
#Train the model
model.fit(x_train,y_train,          
          validation_data=(x_val,y_val),
          epochs=10,
          batch_size=128)

Train on 7666 samples, validate on 2556 samples
Epoch 1/10
7666/7666 [==============================] - 35s 5ms/sample - loss: 5.7921 - accuracy: 0.0093 - val_loss: 4.7869 - val_accuracy: 0.0090
Epoch 2/10
7666/7666 [==============================] - 25s 3ms/sample - loss: 4.7900 - accuracy: 0.0130 - val_loss: 4.7861 - val_accuracy: 0.0094
Epoch 3/10
7666/7666 [==============================] - 25s 3ms/sample - loss: 4.7774 - accuracy: 0.0177 - val_loss: 4.7598 - val_accuracy: 0.0121
Epoch 4/10
7666/7666 [==============================] - 25s 3ms/sample - loss: 4.2501 - accuracy: 0.1224 - val_loss: 4.9846 - val_accuracy: 0.0168
Epoch 5/10
7666/7666 [==============================] - 25s 3ms/sample - loss: 2.4509 - accuracy: 0.4613 - val_loss: 6.5852 - val_accuracy: 0.0184
Epoch 6/10
7666/7666 [==============================] - 25s 3ms/sample - loss: 0.7441 - accuracy: 0.8448 - val_loss: 9.7391 - val_accuracy: 0.0215
Epoch 7/10
7666/7666 [==============================] - 25s 3ms/sample

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
img_generator= tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                               width_shift_range=0.2,
                                                               height_shift_range=0.2,
                                                               horizontal_flip=True,
                                                               validation_split=0.25)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
#Build training generator. 
train_generator = img_generator.flow(x_train_data,y_train_data, subset='training',  batch_size=128)

#Build test generator
val_generator = img_generator.flow(x_train_data,y_train_data, subset='validation', batch_size=128)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [0]:
#Saving the best model using model checkpoint callback
model_checkpoint=tf.keras.callbacks.ModelCheckpoint('dogs_cnn.h5', 
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose=1)

In [34]:
model.fit_generator(train_generator,
                    epochs=10,
                    steps_per_epoch= 7666//128,  #Number of training images//batch_size
                    validation_data=val_generator,
                    validation_steps = 2556//128, #Number of validation images//batch_size
                    callbacks = [model_checkpoint]
                    )

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 59 steps, validate for 19 steps
Epoch 1/10
58/59 [============================>.] - ETA: 0s - loss: 5.0578 - accuracy: 0.0113
Epoch 00001: val_accuracy improved from -inf to 0.01028, saving model to dogs_cnn.h5
59/59 [==============================] - 44s 750ms/step - loss: 5.0532 - accuracy: 0.0114 - val_loss: 4.7834 - val_accuracy: 0.0103
Epoch 2/10
58/59 [============================>.] - ETA: 0s - loss: 4.7809 - accuracy: 0.0127
Epoch 00002: val_accuracy did not improve from 0.01028
59/59 [==============================] - 39s 655ms/step - loss: 4.7808 - accuracy: 0.0127 - val_loss: 4.7829 - val_accuracy: 0.0103
Epoch 3/10
58/59 [============================>.] - ETA: 0s - loss: 4.7786 - accuracy: 0.0131
Epoch 00003: val_accuracy improved from 0.01028 to 0.01110, saving model to dogs_cnn.h5
59/59 [==============================] - 44s 737ms/step - 

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
#from tf.keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= tf.keras.applications.vgg16.VGG16(weights=project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                 include_top=False, pooling='avg')

Print the summary of the base_model

In [30]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:
for layer in base_model.layers:
    layer.trainable = False

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [34]:
base_model.output

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 512) dtype=float32>

In [0]:
#Create your own input format (here 3x200x200)
#input = Input(shape=(3,200,200),name = 'image_input')

#Use the generated model 
output_vgg16 = base_model.output

#Add the fully-connected layers 
x = tf.keras.layers.Flatten(name='flatten')(output_vgg16)
x = tf.keras.layers.Dense(1024, activation='relu', name='fc1')(x)
x = tf.keras.layers.Dense(256, activation='relu', name='fc2')(x)
x = tf.keras.layers.Dense(120, activation='softmax', name='predictions')(x)

In [0]:
#Using Keras Model class
final_model = tf.keras.models.Model(inputs=base_model.input, #Pre-trained model input as input layer
                                    outputs=x)

In [37]:
final_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
#Saving the best model using model checkpoint callback
model_checkpoint=tf.keras.callbacks.ModelCheckpoint('dogs_vgg.h5', 
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose=1)

In [45]:
final_model.fit_generator(train_generator,
                    epochs=10,
                    steps_per_epoch= 7666//128,  #Number of training images//batch_size
                    validation_data=val_generator,
                    validation_steps = 2556//128, #Number of validation images//batch_size
                    callbacks = [model_checkpoint]
                    )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 59 steps, validate for 19 steps
Epoch 1/10
58/59 [============================>.] - ETA: 0s - loss: 3.0369 - accuracy: 0.2495
Epoch 00001: val_accuracy did not improve from 0.19285
59/59 [==============================] - 41s 692ms/step - loss: 3.0394 - accuracy: 0.2491 - val_loss: 3.4607 - val_accuracy: 0.1805
Epoch 2/10
58/59 [============================>.] - ETA: 0s - loss: 3.0129 - accuracy: 0.2604
Epoch 00002: val_accuracy did not improve from 0.19285
59/59 [==============================] - 40s 672ms/step - loss: 3.0089 - accuracy: 0.2613 - val_loss: 3.3799 - val_accuracy: 0.1879
Epoch 3/10
58/59 [============================>.] - ETA: 0s - loss: 2.9946 - accuracy: 0.2639
Epoch 00003: val_accuracy improved from 0.19285 to 0.20025, saving model to dogs_vgg.h5
59/59 [==============================] - 40s 686ms/step - loss: 2.9951 - accuracy: 0.2642 - val_loss: 3.3790 - val_accuracy: 0.2002
Epoch 4/10
58/59 [==============